In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import findspark
# findspark.init('/home/jubinsoni/spark-2.1.0-bin-hadoop2.7')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Bitcoin').getOrCreate()

In [ ]:
# from pyspark.context import SparkContext
# from pyspark.sql.session import SparkSession
# sc = SparkContext.getOrCreate()
# spark = SparkSession(sc)
df = spark.read.csv('/content/btc.csv', inferSchema=True, header=True)

In [ ]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']

In [ ]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Dividends: double (nullable = true)
 |-- Stock Splits: double (nullable = true)



In [ ]:
for line in df.head(5):
    print(line, '\n')

Row(Date=datetime.datetime(2014, 9, 17, 0, 0), Open=465.864013671875, High=468.17401123046875, Low=452.4219970703125, Close=457.3340148925781, Volume=21056800, Dividends=0.0, Stock Splits=0.0) 

Row(Date=datetime.datetime(2014, 9, 18, 0, 0), Open=456.8599853515625, High=456.8599853515625, Low=413.10400390625, Close=424.44000244140625, Volume=34483200, Dividends=0.0, Stock Splits=0.0) 

Row(Date=datetime.datetime(2014, 9, 19, 0, 0), Open=424.1029968261719, High=427.8349914550781, Low=384.5320129394531, Close=394.7959899902344, Volume=37919700, Dividends=0.0, Stock Splits=0.0) 

Row(Date=datetime.datetime(2014, 9, 20, 0, 0), Open=394.6730041503906, High=423.2959899902344, Low=389.88299560546875, Close=408.90399169921875, Volume=36863600, Dividends=0.0, Stock Splits=0.0) 

Row(Date=datetime.datetime(2014, 9, 21, 0, 0), Open=408.0849914550781, High=412.4259948730469, Low=393.1809997558594, Close=398.8210144042969, Volume=26580100, Dividends=0.0, Stock Splits=0.0) 



In [ ]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+---------+------------+
|summary|              Open|              High|               Low|             Close|              Volume|Dividends|Stock Splits|
+-------+------------------+------------------+------------------+------------------+--------------------+---------+------------+
|  count|              3080|              3080|              3080|              3080|                3080|     3080|        3080|
|   mean|13000.845239535245|13326.707258977518|12640.061015963864|13007.106122698102|1.647416954993052E10|      0.0|         0.0|
| stddev|16093.393447184968| 16504.24288218773|15619.891745513898| 16089.63585067296|1.985575808636123...|      0.0|         0.0|
|    min|176.89700317382812| 211.7310028076172|171.50999450683594|178.10299682617188|             5914570|      0.0|         0.0|
|    max|      67549.734375|         68789.625|        66382.0625|      67566.828125|     

In [ ]:
from pyspark.sql.functions import format_number

summary = df.describe()
summary.select(summary['summary'],
                  format_number(summary['Open'].cast('float'), 2).alias('Open'),
                  format_number(summary['High'].cast('float'), 2).alias('High'),
                  format_number(summary['Low'].cast('float'), 2).alias('Low'),
                  format_number(summary['Close'].cast('float'), 2).alias('Close'),
                  format_number(summary['Volume'].cast('int'),0).alias('Volume')
                 ).show()

+-------+---------+---------+---------+---------+---------+
|summary|     Open|     High|      Low|    Close|   Volume|
+-------+---------+---------+---------+---------+---------+
|  count| 3,080.00| 3,080.00| 3,080.00| 3,080.00|    3,080|
|   mean|13,000.85|13,326.71|12,640.06|13,007.11|     null|
| stddev|16,093.39|16,504.24|15,619.89|16,089.64|     null|
|    min|   176.90|   211.73|   171.51|   178.10|5,914,570|
|    max|67,549.73|68,789.62|66,382.06|67,566.83|     null|
+-------+---------+---------+---------+---------+---------+



In [ ]:
df_hv = df.withColumn('HV Ratio', df['High']/df['Volume']).select(['HV Ratio'])
df_hv.show()

+--------------------+
|            HV Ratio|
+--------------------+
|2.223386322852801...|
|1.324876999093942...|
|1.128265760159173...|
|1.148276321331162...|
|1.551634474185751...|
|1.686516624560345...|
|9.790729538818336E-6|
|1.423913645202606...|
|1.579449806871203E-5|
|1.933469340757334...|
|  2.7055350937159E-5|
|1.698267494551865...|
|1.185348497078735...|
|1.126497854186565...|
|1.492138584194584...|
|1.770145650263084...|
|1.222266472901436...|
|7.716215204592186E-6|
|4.102854479750046E-6|
|4.368132388903263E-6|
+--------------------+
only showing top 20 rows



In [ ]:
df.orderBy(df['High'].desc()).select(['Date']).head(1)[0]['Date']

datetime.datetime(2021, 11, 10, 0, 0)

In [ ]:
from pyspark.sql.functions import mean

df.select(mean('Close')).show()

+------------------+
|        avg(Close)|
+------------------+
|13007.106122698102|
+------------------+



In [ ]:
from pyspark.sql.functions import min, max

df.select(max('Volume'),min('Volume')).show()

+------------+-----------+
| max(Volume)|min(Volume)|
+------------+-----------+
|350967941479|    5914570|
+------------+-----------+



In [ ]:
df.filter(df['Close'] < 1370).count()

957

In [ ]:
df.filter('High > 1370').count() * 100/df.count()

68.92857142857143

In [ ]:
from pyspark.sql.functions import corr

df.select(corr(df['High'], df['Volume'])).show()

+------------------+
|corr(High, Volume)|
+------------------+
|0.7208807144018484|
+------------------+



In [ ]:
from pyspark.sql.functions import (dayofmonth, hour,
                                   dayofyear, month,
                                   year, weekofyear,
                                  format_number, date_format)

year_df = df.withColumn('Year', year(df['Date']))

year_df.groupBy('Year').max()['Year', 'max(High)'].show()

+----+------------------+
|Year|         max(High)|
+----+------------------+
|2018|   17712.400390625|
|2015|   495.56201171875|
|2023|     25134.1171875|
|2022|     48086.8359375|
|2014|468.17401123046875|
|2019|  13796.4892578125|
|2020|   29244.876953125|
|2016|   979.39697265625|
|2017|           20089.0|
|2021|         68789.625|
+----+------------------+



In [ ]:
#Create a new column Month from existing Date column
month_df = df.withColumn('Month', month(df['Date']))

#Group by month and take average of all other columns
month_df = month_df.groupBy('Month').mean()

#Sort by month
month_df = month_df.orderBy('Month')

#Display only month and avg(Close), the desired columns
month_df['Month', 'avg(Close)'].show()

+-----+------------------+
|Month|        avg(Close)|
+-----+------------------+
|    1| 13653.07961554852|
|    2|14663.487667747837|
|    3|14833.352723890735|
|    4|15119.381880633036|
|    5| 13220.94630475198|
|    6| 11180.57880007426|
|    7|10856.268967105496|
|    8|12722.796311532298|
|    9|11565.811348742387|
|   10|12331.016559887958|
|   11|   13100.038704088|
|   12|12898.391135376414|
+-----+------------------+

